Spain is the european country with more delay hours per fly. (1)


The Barcelona airport is the airport with less delays hours in Spain. (2)


Vueling is the air carriers with more delays (3)


The shorter the flight is, the more is likely that it will be delay. (4)


The average delay time is Spain is longer than in Germany. (5)


Your are more likely to have a delay flight because of the carrier rather than from weather conditions.

Legacy carriers flights are less likely to be delay than low cost carriers flights.

US airlines are less likely to be delay than the european counterparts.

In [20]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime

In [79]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


In [80]:
response = req.get(url)

In [81]:
response

<Response [200]>

In [114]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


In [115]:
response = req.get(url)

In [116]:
response

<Response [200]>

In [117]:
data_test = response.json()

In [119]:
len(data_test)

367

In [82]:
data_2 = response.json()

In [19]:
len(data)

344

In [24]:
data[0]

{'type': 'departure',
 'status': 'landed',
 'departure': {'iataCode': 'BCN',
  'icaoCode': 'LEBL',
  'terminal': '1',
  'gate': 'B59',
  'baggage': None,
  'delay': '26',
  'scheduledTime': '2019-07-24T11:55:00.000',
  'estimatedTime': '2019-07-24T12:21:00.000',
  'actualTime': '2019-07-24T12:21:00.000',
  'estimatedRunway': '2019-07-24T12:30:00.000',
  'actualRunway': '2019-07-24T12:30:00.000'},
 'arrival': {'iataCode': 'MLA',
  'icaoCode': 'LMML',
  'terminal': None,
  'gate': None,
  'baggage': None,
  'delay': '26',
  'scheduledTime': '2019-07-24T14:00:00.000',
  'estimatedTime': '2019-07-24T14:26:00.000',
  'actualTime': '2019-07-24T14:26:00.000',
  'estimatedRunway': '2019-07-24T14:21:00.000',
  'actualRunway': '2019-07-24T14:21:00.000'},
 'airline': {'name': 'Qatar Airways', 'iataCode': 'QR', 'icaoCode': 'QTR'},
 'flight': {'number': '3682', 'iataNumber': 'QR3682', 'icaoNumber': 'QTR3682'},
 'codeshared': {'airline': {'name': 'Vueling',
   'iataCode': 'VY',
   'icaoCode': 'VLG'}

In [97]:
print(json.dumps(data_2[0], indent = 4, sort_keys=True))

{
    "airline": {
        "iataCode": "BA",
        "icaoCode": "BAW",
        "name": "British Airways"
    },
    "arrival": {
        "actualRunway": "2019-07-24T13:59:00.000",
        "actualTime": "2019-07-24T14:06:00.000",
        "baggage": null,
        "delay": "11",
        "estimatedRunway": "2019-07-24T13:59:00.000",
        "estimatedTime": "2019-07-24T14:06:00.000",
        "gate": null,
        "iataCode": "MXP",
        "icaoCode": "LIMC",
        "scheduledTime": "2019-07-24T13:55:00.000",
        "terminal": "1"
    },
    "codeshared": {
        "airline": {
            "iataCode": "VY",
            "icaoCode": "VLG",
            "name": "Vueling"
        },
        "flight": {
            "iataNumber": "VY6334",
            "icaoNumber": "VLG6334",
            "number": "6334"
        }
    },
    "departure": {
        "actualRunway": "2019-07-24T12:47:00.000",
        "actualTime": "2019-07-24T12:19:00.000",
        "baggage": null,
        "delay": "4",
        

In [ ]:
data_df = pd.DataFrame(data_2)

##### Airline

In [176]:
data_df["airline_name"] = data_df["airline"].apply(lambda x: x['name'])

In [177]:
data_df["airline_code"] = data_df["airline"].apply(lambda x: x['iataCode'])

#### Arrivals

In [178]:
data_df['arrival_delay'] = data_df['arrival'].apply(lambda x: x['delay'])

In [179]:
data_df['arrival_airport'] = data_df['arrival'].apply(lambda x: x['iataCode'])

In [180]:
data_df['schedule_arrival_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])

#### codeshare

In [181]:
data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)

#### Departure

In [182]:
data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])

In [183]:
data_df['depart_airport'] = data_df['departure'].apply(lambda x: x['iataCode'])

In [184]:
data_df['schedule_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])

In [185]:
data_df['departure_gate'] = data_df['departure'].apply(lambda x: x['gate'])

In [186]:
data_df['departure_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])

#### Flight

In [187]:
data_df['icaoNumber'] = data_df['flight'].apply(lambda x: x['icaoNumber'])

In [188]:
data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])

In [189]:
data_df.head(1)

,airline,arrival,codeshared,departure,flight,status,type,airline_name,airline_code,arrival_delay,arrival_airport,schedule_arrival_time,depart_delay,depart_airport,schedule_depart_time,departure_gate,departure_terminal,icaoNumber
0,"{'name': 'British Airways', 'iataCode': 'BA', ...","{'iataCode': 'MXP', 'icaoCode': 'LIMC', 'termi...",Vueling,"{'iataCode': 'BCN', 'icaoCode': 'LEBL', 'termi...",8842,landed,departure,British Airways,BA,11,MXP,2019-07-24T13:55:00.000,4,BCN,2019-07-24T12:15:00.000,A4,1,BAW8842


#### Drop columns

In [191]:
data_df = data_df.drop(['airline','arrival','departure'], axis =1)